# Muhammad Nabil Bin Zulkefli - City FM Data Analyst Assessment 19th July 2024

## Data Pre-Processing

In [ ]:
import pandas as pd

In [ ]:
raw_data = pd.read_csv('/Users/nabilzl/Documents/Nabil/Career/Latest/Companies/City FM/rawdata.csv')

In [ ]:
raw_data.head()

In [ ]:
df = raw_data.copy()

In [ ]:
year = []
month = []
dayoftheweek = []

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'], format='%d/%m/%Y %H:%M:%S')

In [ ]:
for n in range(df.shape[0]):
    year.append(df['datetime'][n].year)
    month.append(df['datetime'][n].month)
    dayoftheweek.append(df['datetime'][n].weekday())

In [ ]:
df['year'] = year
df['month'] = month
df['dayoftheweek'] = dayoftheweek

In [ ]:
df_date_mod = df.copy()

In [ ]:
peaktime= []

In [ ]:
for n in range(df_date_mod.shape[0]):
    if df_date_mod['month'][n] >= 3 and df_date_mod['month'][n] <= 11:
        if df_date_mod['dayoftheweek'][n] >= 0 and df_date_mod['dayoftheweek'][n] <= 4:
            if str(df_date_mod['datetime'][n].time()) > '14:59::00' and str(df_date_mod['datetime'][n].time()) < '21:01:00':
                peaktime.append(1)
            else:
                peaktime.append(0)
        else:
            peaktime.append(0)
    else:
        peaktime.append(0)

In [ ]:
df_date_mod['peaktime'] = peaktime

In [ ]:
df_mod = df_date_mod.copy()

In [ ]:
df_mod.columns

In [ ]:
update_column = ['datetime', 'year', 'month', 'dayoftheweek', 'kWh', 'kVA Demand', 'peaktime']

In [ ]:
df_mod = df_mod[update_column]

In [ ]:
df_mod.to_excel('data_modified.xlsx')

In [ ]:
ea305_n_e_p_w_gst = []
ea305_n_e_p_wo_gst = []
ea310_n_e_p_w_gst = []
ea310_n_e_p_wo_gst = []

In [ ]:
for i in range(df_mod.shape[0]):
    if df_mod['peaktime'][i] == 1:
        ea305_n_e_p_w_gst.append(df_mod['kWh'][i]*0.075655)
        ea305_n_e_p_wo_gst.append(df_mod['kWh'][i]*0.068777)
        ea310_n_e_p_w_gst.append(df_mod['kWh'][i]*0.057380)
        ea310_n_e_p_wo_gst.append(df_mod['kWh'][i]*0.052164)
    else:
        ea305_n_e_p_w_gst.append(df_mod['kWh'][i]*0.016754)
        ea305_n_e_p_wo_gst.append(df_mod['kWh'][i]*0.015231)
        ea310_n_e_p_w_gst.append(df_mod['kWh'][i]*0.015640)
        ea310_n_e_p_wo_gst.append(df_mod['kWh'][i]*0.014218)

In [ ]:
df_mod['EA305_NEP_w_gst'] = ea305_n_e_p_w_gst
df_mod['EA305_NEP_wo_gst'] = ea305_n_e_p_wo_gst
df_mod['EA310_NEP_w_gst'] = ea310_n_e_p_w_gst
df_mod['EA310_NEP_wo_gst'] = ea310_n_e_p_wo_gst

In [ ]:
ea305_supply_charge_w_gst = []
ea305_supply_charge_wo_gst = []
ea310_supply_charge_w_gst = []
ea310_supply_charge_wo_gst = []

In [ ]:
for i in range(df_mod.shape[0]):
    ea305_supply_charge_w_gst.append(df_mod['kVA Demand'][i]*0.543839)
    ea310_supply_charge_w_gst.append(df_mod['kVA Demand'][i]*0.567816)
    ea305_supply_charge_wo_gst.append(df_mod['kVA Demand'][i]*0.494399)
    ea310_supply_charge_wo_gst.append(df_mod['kVA Demand'][i]*0.516195)

In [ ]:
df_mod['EA305_supp_charge_w_gst'] = ea305_supply_charge_w_gst
df_mod['EA305_supp_charge_wo_gst'] = ea305_supply_charge_wo_gst
df_mod['EA310_supp_charge_w_gst'] = ea310_supply_charge_w_gst
df_mod['EA310_supp_charge_wo_gst'] = ea310_supply_charge_wo_gst

In [ ]:
df_mod2 = df_mod.copy()

In [ ]:
df_mod2.to_excel('data_modified.xlsx')

## Network Energy Price Calculation

In [ ]:
date = []
tariff = []
cat = ['ea305','ea310']

In [ ]:
for j in range(2):
    for i in range(len(pd.to_datetime(df_mod2['datetime']).dt.date.unique())):
        date.append(pd.to_datetime(df_mod2['datetime']).dt.date.unique()[i])
        tariff.append(cat[j])

In [ ]:
df_calculation = pd.DataFrame()

In [ ]:
df_calculation['date'] = date
df_calculation['tariff'] = tariff

In [ ]:
df_calculation = df_calculation.sort_values(by=['date','tariff'], ignore_index=True)

In [ ]:
peak_dollar_w_gst = []
peak_dollar_wo_gst = []
peak_consumption = []
offpeak_consumption = []
offpeak_dollar_w_gst = []
offpeak_dollar_wo_gst = []

peak_con_sum = 0.00
offpeak_con_sum = 0.00
peak_gst_dollar_sum = 0.00
peak_non_gst_dollar_sum = 0.00
offpeak_gst_dollar_sum = 0.00
offpeak_non_gst_dollar_sum = 0.00

In [ ]:
for i in range(df_calculation.shape[0]):
    for j in range(df_mod2.shape[0]):
        if df_calculation['date'][i] == pd.to_datetime(df_mod2['datetime'][j]).date():
            if df_calculation['tariff'][i] == 'ea305':
                if df_mod2['peaktime'][j] == 1:
                    peak_con_sum += df_mod2['kWh'][j]
                    peak_gst_dollar_sum += df_mod2['kWh'][j]*0.075655
                    peak_non_gst_dollar_sum += df_mod2['kWh'][j]*0.068777
                else:
                    offpeak_con_sum += df_mod2['kWh'][j]
                    offpeak_gst_dollar_sum += df_mod2['kWh'][j]*0.016754
                    offpeak_non_gst_dollar_sum += df_mod2['kWh'][j]*0.015321
            else:
                if df_mod2['peaktime'][j] == 1:
                    peak_con_sum += df_mod2['kWh'][j]
                    peak_gst_dollar_sum += df_mod2['kWh'][j]*0.057380
                    peak_non_gst_dollar_sum += df_mod2['kWh'][j]*0.052164
                else:
                    offpeak_con_sum += df_mod2['kWh'][j]
                    offpeak_gst_dollar_sum += df_mod2['kWh'][j]*0.015640
                    offpeak_non_gst_dollar_sum += df_mod2['kWh'][j]*0.014218
    
    peak_consumption.append(round(peak_con_sum,2))
    offpeak_consumption.append(round(offpeak_con_sum,2))    
    peak_dollar_w_gst.append(round(peak_gst_dollar_sum,2))
    peak_dollar_wo_gst.append(round(peak_non_gst_dollar_sum,2))
    offpeak_dollar_w_gst.append(round(offpeak_gst_dollar_sum,2))
    offpeak_dollar_wo_gst.append(round(offpeak_non_gst_dollar_sum,2))
    peak_con_sum = 0.00
    peak_gst_dollar_sum = 0.00
    peak_non_gst_dollar_sum = 0.00
    offpeak_con_sum = 0.00
    offpeak_gst_dollar_sum = 0.00
    offpeak_non_gst_dollar_sum = 0.00

In [ ]:
df_calculation['peak_consumption'] = peak_consumption
df_calculation['peak_dollar_w_gst'] = peak_dollar_w_gst
df_calculation['peak_dollar_wo_gst'] = peak_dollar_wo_gst
df_calculation['offpeak_consumption'] = offpeak_consumption
df_calculation['offpeak_dollar_w_gst'] = offpeak_dollar_w_gst
df_calculation['offpeak_dollar_wo_gst'] = offpeak_dollar_wo_gst

In [ ]:
df_calculation.to_excel('df_calculation.xlsx')

## Capacity Charge Calculation

In [ ]:
capacity_charge_w_gst = []
capacity_charge_wo_gst = []
kVA_demand = []

kVA = 0.00
gst_sum_charge = 0.00
non_gst_sum_charge = 0.00

In [ ]:
highest_peak_kVA = df_mod2['kVA Demand'][0]

for i in range(df_calculation.shape[0]):
    for j in range(df_mod2.shape[0]):
        if df_calculation['date'][i] == pd.to_datetime(df_mod2['datetime'][j]).date():
            if df_mod2['peaktime'][j] == 1:
                if df_mod2['kVA Demand'][j] > highest_peak_kVA:
                    highest_peak_kVA = df_mod2['kVA Demand'][j]
                    
    kVA_demand.append(round(highest_peak_kVA,2))
    if df_calculation['tariff'][i] == 'ea305':
        capacity_charge_w_gst.append(round(highest_peak_kVA*0.543839,2))
        capacity_charge_wo_gst.append(round(highest_peak_kVA*0.494399,2))
    else:
        capacity_charge_w_gst.append(round(highest_peak_kVA*0.567815,2))
        capacity_charge_wo_gst.append(round(highest_peak_kVA*0.516195,2))

In [ ]:
df_calculation['kVA_demand'] = kVA_demand
df_calculation['capacity_charge_w_gst'] = capacity_charge_w_gst
df_calculation['capacity_charge_wo_gst'] = capacity_charge_wo_gst

In [ ]:
df_calculation.to_excel('df_calculation.xlsx')

## Supply Charge Calculation

In [ ]:
supply_charge_w_gst = []
supply_charge_wo_gst = []

In [ ]:
for i in range(df_calculation.shape[0]):
    if df_calculation['tariff'][i] == 'ea305':
        supply_charge_w_gst.append(round(22.517477,2))
        supply_charge_wo_gst.append(round(20.470434,2))
    else:
        supply_charge_w_gst.append(round(45.222690,2))
        supply_charge_wo_gst.append(round(41.111536,2))

In [ ]:
df_calculation['supply_charge_w_gst'] = supply_charge_w_gst
df_calculation['supply_charge_wo_gst'] = supply_charge_wo_gst

In [ ]:
df_calculation.to_excel('data_processed.xlsx')